In [1]:
import programs.new_builder as nb
from pmagpy import ipmag
import os
import json
import numpy as np
import sys

import pandas as pd
from pandas import DataFrame
from pmagpy import builder
from pmagpy import validate_upload
from pmagpy import pmag

pmagpy_dir = os.path.join(os.getcwd(), 'pmagpy')
if pmagpy_dir not in sys.path:
    sys.path.append(pmagpy_dir)
    
WD = os.path.join(os.getcwd(), "3_0", "McMurdo")


In [2]:
# get specimen level data from specimen table
contribution = nb.Contribution(WD, read_tables=['measurements', 'specimens', 'samples','sites'])
if 'specimens' in contribution.tables:
    spec_container = contribution.tables['specimens']
    spec_data = spec_container.df
spec_data.columns


Index([u'analyst_names', u'aniso_ftest', u'aniso_ftest12', u'aniso_ftest23',
       u'aniso_s', u'aniso_s_mean', u'aniso_s_n_measurements',
       u'aniso_s_sigma', u'aniso_s_unit', u'aniso_tilt_correction',
       u'aniso_type', u'aniso_v1', u'aniso_v2', u'aniso_v3', u'citations',
       u'description', u'dir_alpha95', u'dir_comp_name', u'dir_dec',
       u'dir_inc', u'dir_mad_free', u'dir_n_measurements',
       u'dir_tilt_correction', u'experiment_names', u'geologic_classes',
       u'geologic_types', u'hyst_bc', u'hyst_bcr', u'hyst_mr_moment',
       u'hyst_ms_moment', u'int_abs', u'int_b', u'int_b_beta', u'int_b_sigma',
       u'int_corr', u'int_dang', u'int_drats', u'int_f', u'int_fvds',
       u'int_gamma', u'int_mad_free', u'int_md', u'int_n_measurements',
       u'int_n_ptrm', u'int_q', u'int_rsc', u'int_treat_dc_field',
       u'lithologies', u'magic_method_codes', u'meas_step_max',
       u'meas_step_min', u'meas_step_unit', u'method_codes', u'sample',
       u'software_pack

In [3]:
# find the record of a particular specimen.  if it already has aniso data, then modify in place
# otherwise add a new record and modify that....    
specimen='mc121d1' # a specimen with aniso data
new_s='.3:.3:.3:0:0:0'
if len(spec_data[spec_data['specimen'].str.contains(specimen) & spec_data['aniso_s'].notnull()])>0:
    #print spec_data[spec_data['specimen'].str.contains(specimen)==True]
    spec_data['aniso_s']=np.where(spec_data['specimen'].str.contains(specimen) & spec_data['aniso_s'].notnull(), new_s,spec_data['aniso_s'])
else:
    print 'no record found - creating new one'
print spec_data[spec_data['specimen'].str.contains(specimen)==True]

         analyst_names aniso_ftest aniso_ftest12 aniso_ftest23  \
specimen                                                         
mc121d1    @jsteindorf         3.8           0.4           5.3   
mc121d1        @ltauxe        None          None          None   
mc121d1        @ltauxe        None          None          None   

                 aniso_s aniso_s_mean aniso_s_n_measurements aniso_s_sigma  \
specimen                                                                     
mc121d1   .3:.3:.3:0:0:0   0.00005973                      9      0.013881   
mc121d1             None         None                   None          None   
mc121d1             None         None                   None          None   

         aniso_s_unit aniso_tilt_correction   ...    int_treat_dc_field  \
specimen                                      ...                         
mc121d1          Am^2                    -1   ...                  None   
mc121d1          None                  None   ...    